# Replication Study: Automated Player Tracking with YOLOv8

**Author:** Parau Raul

**Course:** Knowledge Based Systems

**Objective:** Partial replication of the tracking-by-detection pipeline described by Muller et al. (2024), employing the updated YOLOv8 architecture on consumer-grade hardware.

## 1. Environment Setup
We utilize `ultralytics` for the object detection model, `opencv` for video processing, and `pandas/matplotlib` for analyzing the performance metrics.

In [1]:
# Install necessary packages (uncomment if running in Colab or fresh env)
# pip install ultralytics opencv-python pandas matplotlib
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO
from collections import defaultdict
import numpy as np
from datetime import datetime
import os


OSError: [WinError 1114] A dynamic link library (DLL) initialization routine failed. Error loading "C:\Users\parau\anaconda3\Lib\site-packages\torch\lib\c10.dll" or one of its dependencies.

## 2. Tracking Methodology
We implement a **Tracking-by-Detection** loop.
- **Detector:** YOLOv8m (Medium) pre-trained on COCO.
- **Tracker:** BoT-SORT (integrated) for ID persistence.
- **Visualization:** We compute and draw trajectory tails (last 30 frames) for each unique player ID to visualize movement patterns.

In [ ]:
def run_tracker(source_video, output_video, model_type='yolov8m.pt'):
    # Load the model
    model = YOLO(model_type)
    
    # Video Setup
    cap = cv2.VideoCapture(source_video)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Output Setup
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))
    
    # CSV Logger Setup
    csv_data = []
    
    # Trajectory History
    track_history = defaultdict(lambda: [])
    
    print(f"Starting inference on {source_video}...")
    print(f"Model: {model_type} | Total Frames: {total_frames}")

    frame_idx = 0
    
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break
            
        start_time = datetime.now()
        
        # --- INFERENCE ---
        # Classes: 0=Person, 32=Sports ball
        results = model.track(frame, persist=True, conf=0.25, classes=[0, 32], verbose=False)
        
        # --- VISUALIZATION ---
        annotated_frame = results[0].plot()
        
        if results[0].boxes.id is not None:
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            
            # Plot Trajectories
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y))) 
                if len(track) > 40:  # Keep last 40 frames
                    track.pop(0)

                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(annotated_frame, [points], isClosed=False, color=(0, 255, 255), thickness=2)
        
        # --- LOGGING ---
        inference_ms = (datetime.now() - start_time).total_seconds() * 1000
        player_count = sum(1 for c in results[0].boxes.cls if c == 0)
        ball_detected = 1 if 32 in results[0].boxes.cls else 0
        
        csv_data.append({
            'Frame': frame_idx,
            'Inference_Time_ms': inference_ms,
            'FPS_Instant': 1000/inference_ms if inference_ms > 0 else 0,
            'Players_Visible': player_count,
            'Ball_Visible': ball_detected
        })
        
        out.write(annotated_frame)
        frame_idx += 1
        
        if frame_idx % 50 == 0:
            print(f"Processed {frame_idx}/{total_frames} frames...", end='\r')

    cap.release()
    out.release()
    
    # Save Statistics
    df = pd.DataFrame(csv_data)
    csv_name = output_video.replace('.mp4', '_stats.csv')
    df.to_csv(csv_name, index=False)
    
    print(f"\nDone! Video saved to {output_video}")
    print(f"Stats saved to {csv_name}")
    return df, csv_name

## 3. Experiment Execution
We define the input source (a sample clip from SoccerNet) and run the pipeline.
*Note: Ensure `sample_match.mp4` exists in the directory.*

In [ ]:
# Define paths
input_video = "sample1.mp4"  # <--- SCHIMBA AICI CU NUMELE VIDEO-ULUI TAU
output_video = "replication_result.mp4"

# Run the experiment
if os.path.exists(input_video):
    df_results, csv_path = run_tracker(input_video, output_video, model_type='yolov8m.pt')
else:
    print("Error: Video file not found. Please upload a video.")

## 4. Results & Discussion
We analyze the stability and performance of the model using the logged data.
The graphs below demonstrate the real-time capability (FPS) and the tracking density.

In [ ]:
# Plotting the results
plt.figure(figsize=(14, 6))

# Subplot 1: Processing Speed
plt.subplot(1, 2, 1)
plt.plot(df_results['Frame'], df_results['FPS_Instant'], color='green', label='Instant FPS')
plt.axhline(y=30, color='r', linestyle='--', label='Real-Time Threshold (30 FPS)')
plt.title('Inference Speed Analysis (YOLOv8m)')
plt.xlabel('Frame Number')
plt.ylabel('FPS')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 2: Player Count Stability
plt.subplot(1, 2, 2)
plt.plot(df_results['Frame'], df_results['Players_Visible'], color='blue', alpha=0.7)
plt.title('Player Tracking Consistency')
plt.xlabel('Frame Number')
plt.ylabel('Detected Players')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate averages
avg_fps = df_results['FPS_Instant'].mean()
avg_players = df_results['Players_Visible'].mean()
ball_detection_rate = (df_results['Ball_Visible'].sum() / len(df_results)) * 100

print(f"=== Experiment Summary ===")
print(f"Average FPS: {avg_fps:.2f}")
print(f"Average Players Tracked: {avg_players:.1f}")
print(f"Ball Detection Rate: {ball_detection_rate:.1f}%")